In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import numpy as np 
import requests
import matplotlib.pyplot as plt
import os

## Scraper un article ICML

In [12]:
original_url = "https://icml.cc"
complement_url = "/virtual/2024/papers.html?filter=titles"

content_papers_page = requests.get(original_url+complement_url).text
scraper = BeautifulSoup(content_papers_page, parser="lxml")

papers = scraper.find("noscript")
title_papers = papers.find_all("li")

icml_papers = {
    "Year" : [], 
    "Title" : [], 
    "Authors" : [],
    "Number of authors" : [],
    "Abstract" : [],
    "Citations" : [],
    "Paper file" : []
}

for title_paper in title_papers :
    
    icml_papers["Year"].append(2024)
    icml_papers['Citations'].append(0)
    # title of the paper
    title = title_paper.a.text
    
    icml_papers["Title"].append(title)
    
    relative_link_to_detail = title_paper.a.get("href")
    
    url_to_detail = original_url + relative_link_to_detail
    
    content_detail_page = requests.get(url_to_detail).text
    scraper2 = BeautifulSoup(content_detail_page, parser="lxml")
    
    # authors
    authors = scraper2.find("h3", class_="card-subtitle mb-2 text-muted text-center").text[2:]
    authors = authors.strip()
    authors = authors.replace("·", ",")
    
    icml_papers["Authors"].append(authors)
    icml_papers["Number of authors"].append(len(authors.split(",")))
    
    content_to_project = scraper2.find("div", class_="text-center p-4")
    
    # project url (à voir si ca nous pose pas de problème avec les projets qui n'ont pas de page projet et donc peut être utiliser un autre moyen d'avoir le a)
    # project_url = content_to_project.find_all("a")[1].get("href")

    # abstract
    try : # ce qui ont une page projet vient avant l'abstract 
        abstract_paper_pdf_page = content_to_project.find_all("a")[2].get("href")
    except : 
        abstract_paper_pdf_page = content_to_project.find_all("a")[1].get("href")
    
    if abstract_paper_pdf_page[-3:] == "pdf" : 
        link_to_pdf = abstract_paper_pdf_page
        abstract = "" # trouver une solution pour le retrouver
        icml_papers["Abstract"].append(abstract)
        icml_papers["Paper file"].append(link_to_pdf)
        continue
    else : 
        try : # missing schema
            content_project = requests.get(abstract_paper_pdf_page).text
        except :
            abstract_paper_pdf_page = content_to_project.find_all("a")[1].get("href")
            content_project = requests.get(abstract_paper_pdf_page).text
        
        scraper3 = BeautifulSoup(content_project, parser="lxml")

        try : 
            abstract = scraper3.find("div", id="abstract").text
        except AttributeError as e : 
            try : 
                abstract = scraper3.find("p", class_="abstract").text
            except : 
                abstract = ""
                link_to_pdf = ""
                
                icml_papers["Abstract"].append(abstract)
                icml_papers["Paper file"].append(link_to_pdf)
                continue # to skip the rest of the loop
            
        abstract = abstract.strip()

        # link to PDF
        try :
            link_to_pdf = scraper3.find("div", id="extras").li.a.get("href")
        except AttributeError as e : 
            link_to_pdf = scraper3.find("a", id="pdf").get("href")
        
        icml_papers["Abstract"].append(abstract)
        icml_papers["Paper file"].append(link_to_pdf)
    
    print(title, authors, abstract, link_to_pdf)

Position: Towards Unified Alignment Between Agents, Humans, and Environment Zonghan Yang , an liu , Zijun Liu , Kaiming Liu , Fangzhou Xiong , Yile Wang , Zeyuan Yang , Qingyuan Hu , XinRui Chen , Zhenhe Zhang , Fuwen Luo , Zhicheng Guo , Peng Li , Yang Liu The rapid progress of foundation models has led to the prosperity of autonomous agents, which leverage the universal capabilities of foundation models to conduct reasoning, decision-making, and environmental interaction. However, the efficacy of agents remains limited when operating in intricate, realistic environments. In this work, we introduce the principles of Unified Alignment for Agents (UA$^2$), which advocate for the simultaneous alignment of agents with human intentions, environmental dynamics, and self-constraints such as the limitation of monetary budgets. From the perspective of UA$^2$, we review the current agent research and highlight the neglected factors in existing agent benchmarks and method candidates. We also con

In [11]:
stringtest = '/media/icml-2024/Slides/35641.pdf'
stringtest[-3:]

'pdf'